<a href="https://colab.research.google.com/github/OrsonXu/CSNext2022_LEG/blob/main/csnext_legmove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from sklearn.preprocessing import RobustScaler
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Dropout, MaxPooling1D, Flatten
from scipy import stats
import numpy as np
import tensorflow as tf
from tensorflow import keras
import seaborn as sns


In [15]:
csvs = []
_df   = pd.read_csv('data/NOISE_1_1.csv', names = [('timestamp','linearacc_x','linearacc_y','linearacc_z','gyro_x','gyro_y','gyro_z','batch,interval')])

# each user files
# filename should be: SHAKING_<num_test>_<user_id>.csv or NOISE_<num_test>_<user_id>.csv
for j in range(1):

    # read shaking files
    for i in range(3):    
        _df = pd.read_csv(f'data/SHAKING_{i+1}_{j+1}.csv')
        _df = _df.drop(columns=['gyro_x', 'gyro_y', 'gyro_z', 'batch', 'interval'])
        _df["label"] = "shaking" # add label
        _df["user_id"] = j+1
        #print(_df.head(3))        
        csvs.append(_df)

    # read noise files
    for i in range(5):
        _df = pd.read_csv(f'data/NOISE_{i+1}_{j+1}.csv')
        _df = _df.drop(columns=['gyro_x', 'gyro_y', 'gyro_z', 'batch', 'interval'])
        _df["label"] = "noise" # add label
        _df["user_id"] = j+1
        # print(_df.head(3))        
        csvs.append(_df)

print(f"{len(csvs)} files read!")

# concatenate all csvs in one dataframe
df = pd.concat(csvs)
df['linearacc_xyz'] = np.sqrt(df['linearacc_x'] ** 2 + df['linearacc_y'] ** 2 + df['linearacc_z'] ** 2 )
print(df.shape)
df.sample(frac=0.05, replace=True, random_state=1)

8 files read!
(5835, 7)


,timestamp,linearacc_x,linearacc_y,linearacc_z,label,user_id,linearacc_xyz
57,1650944683689,-0.964006,1.043366,-1.868314,noise,1,2.347023
235,1650944563155,-0.311301,-1.592262,0.790868,shaking,1,1.804904
335,1650944637073,0.022619,0.021746,-0.043715,noise,1,0.053810
92,1650944684273,4.347627,-1.443706,0.567434,noise,1,4.616073
185,1650944591322,2.132645,-4.241833,1.856893,shaking,1,5.097977
...,...,...,...,...,...,...,...
571,1650944661756,0.019331,0.054645,-0.078251,noise,1,0.097381
151,1650944634006,0.025495,0.006613,-0.077078,noise,1,0.081454
221,1650944671174,0.197019,0.259100,-0.126654,noise,1,0.349271
92,1650944653772,0.014831,0.108275,-0.203812,noise,1,0.231264


# Load data

In [12]:
# split data into training and test, 10 users expected
# df_train = df[df['user_id'] <= 6]
# df_test = df[df['user_id'] > 6]

df_train = df[df['user_id'] == 1]
df_test = df[df['user_id'] == 1]

# scale training data
scale_columns = ['linearacc_x', 'linearacc_y', 'linearacc_z']
mapping_dict = {'noise': 0, 'shaking': 1}
scaler = RobustScaler()
scaler = scaler.fit(df_train[scale_columns])

df_train.loc[:, scale_columns] = scaler.transform(df_train[scale_columns].to_numpy())
df_test.loc[:, scale_columns] = scaler.transform(df_test[scale_columns].to_numpy())

# create sequences using windows slicing
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []

    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    ys_int = [mapping_dict[y] for y in ys]
    ys_vector = keras.utils.to_categorical(ys_int)
    return np.array(Xs), ys_vector



TIME_STEPS = 90 # window_size
STEP = 15

X_train, y_train = create_dataset(
    df_train[['linearacc_xyz']],
    df_train.label,
    TIME_STEPS,
    STEP
)

X_test, y_test = create_dataset(
    df_test[['linearacc_xyz']],
    df_test.label,
    TIME_STEPS,
    STEP
)

print(X_train.shape, y_train.shape)
#print(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  "X does not have valid feature names, but"


(383, 90, 1) (383, 2)


In [ ]:
print(y_train)

# **`Model Training `**

In [21]:
# define the keras model 
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(90,1)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(8,activation='relu'))
model.add(Dense(2, activation = 'softmax'))
 # compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
...



Ellipsis

In [22]:
...
# fit the keras model on the dataset
model.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=10)

...

Epoch 1/30
31/31 [==============================] - 1s 14ms/step - loss: 0.4098 - accuracy: 0.8301 - val_loss: 3.8969 - val_accuracy: 0.3506
Epoch 2/30
31/31 [==============================] - 0s 8ms/step - loss: 0.1259 - accuracy: 0.9771 - val_loss: 4.9466 - val_accuracy: 0.4416
Epoch 3/30
31/31 [==============================] - 0s 6ms/step - loss: 0.0237 - accuracy: 0.9902 - val_loss: 5.8217 - val_accuracy: 0.4416
Epoch 4/30
31/31 [==============================] - 0s 6ms/step - loss: 0.0094 - accuracy: 1.0000 - val_loss: 6.1092 - val_accuracy: 0.4416
Epoch 5/30
31/31 [==============================] - 0s 6ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 6.8172 - val_accuracy: 0.4416
Epoch 6/30
31/31 [==============================] - 0s 6ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 7.2437 - val_accuracy: 0.4416
Epoch 7/30
31/31 [==============================] - 0s 6ms/step - loss: 0.0075 - accuracy: 0.9935 - val_loss: 7.5613 - val_accuracy: 0.4286
Epoch 8/30
31/31 [=

Ellipsis

In [ ]:
print(model.summary())
input_shape=[X_train.shape[1], X_train.shape[2]]
print(input_shape)

In [ ]:
# evaluate the keras model
accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

NameError: ignored

In [ ]:
history = model.fit(
    X_train,y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    #shuffle= True
    shuffle = False 
)


Epoch 1/50
11/11 [==============================] - 1s 61ms/step - loss: 0.2892 - acc: 0.8909 - val_loss: 1.2057 - val_acc: 0.2703
Epoch 2/50
11/11 [==============================] - 1s 53ms/step - loss: 0.2759 - acc: 0.9091 - val_loss: 1.0170 - val_acc: 0.4865
Epoch 3/50
11/11 [==============================] - 1s 50ms/step - loss: 0.3072 - acc: 0.9030 - val_loss: 1.2199 - val_acc: 0.2973
Epoch 4/50
11/11 [==============================] - 1s 54ms/step - loss: 0.3036 - acc: 0.9061 - val_loss: 1.3594 - val_acc: 0.2432
Epoch 5/50
11/11 [==============================] - 1s 52ms/step - loss: 0.2620 - acc: 0.9121 - val_loss: 1.1214 - val_acc: 0.3514
Epoch 6/50
11/11 [==============================] - 1s 51ms/step - loss: 0.2962 - acc: 0.9061 - val_loss: 1.1847 - val_acc: 0.3514
Epoch 7/50
11/11 [==============================] - 1s 52ms/step - loss: 0.2705 - acc: 0.9152 - val_loss: 1.3536 - val_acc: 0.2703
Epoch 8/50
11/11 [==============================] - 1s 50ms/step - loss: 0.2753 - a

In [ ]:
#print(y_train)
print(X_train.shape)
print(y_train.shape)
set(y_train[:,0]) # unique value 

(367, 90, 3)
(367, 2)


{0.0, 1.0}

# Evaluation

In [ ]:
model.evaluate(X_test, y_test)